In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib notebook
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px


In [4]:
files = glob.glob('*20200615*.bin.csv')
files.sort()
files = files[::-1]
for file in files:
    print(file.split('_')[1])
    

NIST0005
NIST0004
NIST0003
NIST0001
NIST0000
NIST-GEN


In [80]:
df = pd.read_csv(files[0])
# df.drop((df[df[' epoch_time']<3]).index, inplace=True)
df['# time'] = pd.to_datetime(df['# time'])
df.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
    # cleanup whitespaces in data
df = df.applymap(lambda x: x.strip() if type(x)==str else x)

t3 = pd.to_datetime('2020-06-14 18:00:00')
df.drop((df[df['time']<t3]).index, inplace=True)



In [81]:
hist_id = np.array(np.unique(df['encounter_id'],return_counts=True))
good_id = hist_id[0, hist_id[1,:]>1] 

df_by_id = {}
for id in good_id:
    df_by_id[id] = df[df['encounter_id']==id]
    #print(df_by_id[id])

In [82]:
def plot_df(df):
    labels = ['mean37', 'n37', 'var37']
    labels = ['mean38', 'n38', 'var38']
    labels = ['mean39', 'n39', 'var39']
    
    fig = go.FigureWidget()
    def add(fig, labels, color):
        var = (df[labels[2]]/df[labels[1]])**0.5
        # print(var)
        fig.add_scatter(x = df['time'], y=-df[labels[0]], 
                      error_y = dict(type='data', array=var, visible=True),
                      mode='markers', name=labels[1], 
                      marker = dict(size=5, symbol='circle', color=color)
                       )

                     
    labels = ['mean39', 'n39', 'var39']
    add(fig, labels, 'green')
    labels = ['mean37', 'n37', 'var37']
    add(fig, labels, 'red')
    labels = ['mean38', 'n38', 'var38']
    add(fig, labels, 'blue')
    
    return fig

In [83]:
for i in range(len(good_id)):
    # print(good_id[i])
    fig = plot_df(df_by_id[good_id[i]])
    fig.update_layout(
        title=good_id[i][:8],
        xaxis_title="time",
        yaxis_title="rssi"
    )
    fig.show()

In [89]:
for file in files[:-1]:
    print(file)
    df = pd.read_csv(file)
    # df.drop((df[df[' epoch_time']<3]).index, inplace=True)
    df['# time'] = pd.to_datetime(df['# time'])
    df.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
        # cleanup whitespaces in data
    df = df.applymap(lambda x: x.strip() if type(x)==str else x)

    t3 = pd.to_datetime('2020-06-14 18:00:00')
    df.drop((df[df['time']<t3]).index, inplace=True)



    fig = px.scatter(df, x="time", y="mean39", color="encounter_id")
    for trace in fig.data:
        trace.name = trace.name[:4]
        trace.y = -trace.y
    fig.show()

raw_NIST0005_20200615_085428.bin.csv


raw_NIST0004_20200615_085604.bin.csv


raw_NIST0003_20200615_085724.bin.csv


raw_NIST0001_20200615_084344.bin.csv


raw_NIST0000_20200615_090024.bin.csv


In [64]:
import datetime
t = '2020-06-14 15:47:00'

In [65]:
pd.to_datetime(t)


Timestamp('2020-06-14 15:47:00')

In [70]:
t2 = '2020-06-14 18:00:00'

In [73]:
t3 = pd.to_datetime('2020-06-14 18:00:00')

In [74]:
pd.to_datetime(t) > pd.to_datetime(t2)

False

In [75]:
df['time']>t3

0       False
1       False
2       False
3       False
4       False
        ...  
2645     True
2646     True
2647     True
2648     True
2649     True
Name: time, Length: 2650, dtype: bool

In [77]:
df.drop((df[df['time']<t3]).index, inplace=True)


In [78]:
df


,time,epoch_time,first_sec,last_sec,mean37,n37,min37,max37,var37,mean38,...,max38,var38,mean39,n39,min39,max39,var39,crypto_flag,extra_flag,encounter_id
912,2020-06-14 19:34:00,26536414,5,55,75,7,60,97,1775,72,...,91,1339,74,6,62,82,679,7,0,B1F7C65C8C8F4B3F1CE833DA3F912C7E1F14BB0E27C425...
913,2020-06-14 19:22:00,26536402,0,58,87,6,87,88,6,92,...,93,11,81,6,81,82,1,7,0,B1F7C65C8C8F4B3F1CE833DA3F912C7E1F14BB0E27C425...
914,2020-06-14 19:22:00,26536402,0,58,69,6,68,70,4,74,...,78,44,73,7,73,75,10,7,0,74972B1E1B21DC64DC0573EF71620B205FC211DDAF9E5B...
915,2020-06-14 19:23:00,26536403,0,58,88,8,85,93,133,85,...,89,115,85,9,85,90,53,7,0,6E2909E1C96E78FF4759BC7DA011AC692316F5179047A5...
916,2020-06-14 19:23:00,26536403,0,58,49,8,48,49,5,54,...,54,0,54,9,54,54,0,7,0,65E6B8FDDED06244A1934443D088CC7EB1561675484DD8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,2020-06-14 22:35:00,26536595,0,58,40,8,39,40,2,40,...,41,2,41,9,40,41,5,7,0,B1F7C65C8C8F4B3F1CE833DA3F912C7E1F14BB0E27C425...
2646,2020-06-14 22:35:00,26536595,0,58,45,8,45,45,0,44,...,44,3,43,9,42,44,9,7,0,65E6B8FDDED06244A1934443D088CC7EB1561675484DD8...
2647,2020-06-14 22:35:00,26536595,0,58,59,7,59,59,0,63,...,63,3,68,8,68,68,0,7,0,4DA0EDFBDB272A239664AFC65DFB357F84A9D2681616DC...
2648,2020-06-14 22:35:00,26536595,0,58,57,7,57,57,0,59,...,59,0,63,9,63,63,0,7,0,4B3CCE1AE7426014CDA00E27BB98A91873D5D5A31B33BE...
